In [1]:
import pandas as pd
from astroquery.simbad import Simbad
from jwst import *

%load_ext autoreload
%autoreload 2
Simbad.add_votable_fields('flux(K)','sp')

max_bex_age = 1000 #This isn't the actually maximum age, but the maximum age above which there isn't much data. 

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


Directories to change depending on what computer you're on

In [35]:
figure_base_directory = "./FarOut_Figures/" #Change this to reflect the project we're working on
bex_file = '../models/BEX_evol_mags_-2_MH_0.00AC.dat'

In [3]:
computer = 'connor'
connor = True
maxmb = False

if computer == 'connor':
    import os
    base_model_dir="/Users/connorvancil/Desktop/AstroResearch/Data/models/evolutionary_tracks/ATMO_CEQ/"
    bex_file = '../models/BEX_evol_mags_-2_MH_0.00AC.dat'
    os.environ['PYSYN_CDBS']='/Users/connorvancil/Desktop/AstroResearch/Data/trds/'
elif computer == 'max_laptop': 
    base_model_dir="/Users/maxwellmb/Data/models/ATMO_2020_models/evolutionary_tracks/ATMO_CEQ/"
    bex_file = '../models/BEX_evol_mags_-2_MH_0.00AC.dat'
elif computer == 'max_desktop':
    base_model_dir = "TBD"
    bex_file = None

Read in the Data Table

In [18]:
df = pd.read_csv('../large_orbit_targetlist.csv')

data_arr = df[['hostname','sy_dist','st_age']].to_numpy()
target_dict={}
for row in data_arr:
    target_dict[row[0]]=row[1:]

Query all targets

In [19]:
targets=df['hostname'].to_numpy()

queried_params=[];spts=[]
for target in targets:
    result_table = Simbad.query_object(target)
    spts.append(result_table['SP_TYPE'][0])
    queried_params.append([target,result_table['FLUX_K'][0]])

Simbad doesn't Have these spectral types, so add them manually

In [20]:
spts[10] = 'F5V'
spts[25] = 'F0V'
spts[38] = 'K4'
spts[44] = 'K0'
spts[54] = 'K0V'
spts[56] = 'M5V'
spts[58] = 'G7'
spts[66] = 'K0V'
spts[72] = 'K0'
spts[73] = 'K0'
spts[75] = 'K4'
spts[77] = 'G6'
spts[91] = 'M3'
spts[96] = 'G0'
spts[97] = 'M0'
spts[100] = 'A0'
spts[110] = 'F7'
spts[123] = 'K1'
spts[127] = 'K0'
spts[130] = 'K0V'
spts[131] = 'K0'
spts[132] = 'K0'
spts[134] = 'K8V'
spts[137] = 'G4'

Force Spectral Types to those in ck04models

In [21]:
from models import *
for i,spt in enumerate(spts):
    if spt not in lookuptable: 
        spts[i] = match_spt(spt)
[params.append(spts[i]) for i, params in  enumerate(queried_params)]

#adding another dictionary to hold these queried paramters in a more useful format
simbad_vals={}
for star in queried_params:
    simbad_vals[star[0]]=[star[1],star[2]]

Set up some configuration info

In [22]:
configs = {'f250m':  {'instrument':'nircam','instrument_mask':'mask335r'},
            'f300m': {'instrument':'nircam','instrument_mask':'mask335r'}, 
            'f356w': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f410m': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f444w': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f1140c':{'instrument':'miri',  'instrument_mask':None},
            'f1550c':{'instrument':'miri',  'instrument_mask':None},
            }
filter_list = list(configs.keys())
filter_list = ['f444w'] #For now only doing this filter because the bex filters only have this filter.

In [47]:
full_detection_list = []

for i, target in enumerate(targets):

    age = target_dict[target][1]
    distance = target_dict[target][0]

    kmag,spt = simbad_vals[target]

    print("Target {}: {}, {} Myr, {:.0f} pc, kmag={:.1f}".format(i,target,age,distance,kmag))

    for filter_name in filter_list:
        
        instrument = configs[filter_name]['instrument']
        instrument_mask = configs[filter_name]['instrument_mask']

        jwst_mag = get_jwst_mag(spt,kmag,instrument,filter_name,filter_dir="./",jwst_mask=instrument_mask,
                    norm_filter='bessel_k').value

        contrast_curves=read_contrast_curves()
        sep,companion_mags=companion_detection_limit(jwst_mag,filter_name,contrast_curves,plot=False);

        good_seps = sep[np.where(np.isfinite(companion_mags))]

        model_dir = base_model_dir+model_instrument_directories.get(instrument.lower())
        
        if instrument.lower () == "nircam":
            model_dir+=model_nircam_mask_directories.get(instrument_mask.lower())

        if age < max_bex_age:
            mass_limits = generate_mass_curve(age,distance,companion_mags,filter_name,sep,model_dir,bex_model_filename=bex_file,
                            plot=False)
        else: 
            mass_limits = generate_mass_curve(age,distance,companion_mags,filter_name,sep,model_dir,
                            plot=False)
        onearc_mass=mass_limits[23]

        if ((onearc_mass <= 10) and (onearc_mass > 5)):
            figure_directory = figure_base_directory + 'Sub10MJup/'
        elif ((onearc_mass) <=5 and  (onearc_mass >1)):
            figure_directory = figure_base_directory + 'Sub5MJup/'
        elif onearc_mass <=1:
            figure_directory = figure_base_directory + 'Sub1MJup/'
        else:
            figure_directory = figure_base_directory + 'Others/'

    fig,axis = plt.subplots(1,1,figsize=(12,5))
    plt.semilogy(sep,mass_limits,label=filter_name)
    plt.title("{}, {} Myr, {} pc, kmag={:.1f}, Mass at 1arcsec: {:.1f}".format(target,age,distance,kmag,mass_limits[23]))

    ymin, ymax = axis.get_ylim()

    plt.xlabel("Separation (arcseconds)")
    plt.ylabel(r"Mass Limit ($M_{Jup}$)")
    plt.savefig(figure_directory+target+".png",dpi=200)
    print("Saving to {}".format(figure_directory+target+".png"))
    plt.close()

Target 0: eps Eri, 500 Myr, 3 pc, kmag=1.7


/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(masses)[sorted_by_mass], np.array(age_list)[sorted_by_mass], np.array(mag_list)[sorted_by_mass]
/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:319: RuntimeWarning: Mean of empty slice
  masses = np.nanmean(np.vstack([atmo_mass_interp,bex_mass_interp]),axis=0)


Saving to ./FarOut_Figures/Sub5MJup/eps Eri.png
Target 1: GJ 674, 550 Myr, 4 pc, kmag=4.9


/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(masses)[sorted_by_mass], np.array(age_list)[sorted_by_mass], np.array(mag_list)[sorted_by_mass]
/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:319: RuntimeWarning: Mean of empty slice
  masses = np.nanmean(np.vstack([atmo_mass_interp,bex_mass_interp]),axis=0)


Saving to ./FarOut_Figures/Others/GJ 674.png
Target 2: AU Mic, 22 Myr, 10 pc, kmag=4.5


...

Saving to ./FarOut_Figures/Sub10MJup/HD 56414.png
Target 136: WASP-135, 600 Myr, 298 pc, kmag=11.0


/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(masses)[sorted_by_mass], np.array(age_list)[sorted_by_mass], np.array(mag_list)[sorted_by_mass]
/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:319: RuntimeWarning: Mean of empty slice
  masses = np.nanmean(np.vstack([atmo_mass_interp,bex_mass_interp]),axis=0)


Saving to ./FarOut_Figures/Sub5MJup/WASP-135.png
Target 137: Kepler-1312, 400 Myr, 299 pc, kmag=10.9


/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(masses)[sorted_by_mass], np.array(age_list)[sorted_by_mass], np.array(mag_list)[sorted_by_mass]
/Users/connorvancil/Desktop/AstroResearch/contraster/testing/models.py:319: RuntimeWarning: Mean of empty slice
  masses = np.nanmean(np.vstack([atmo_mass_interp,bex_mass_interp]),axis=0)


Saving to ./FarOut_Figures/Sub5MJup/Kepler-1312.png
